## Importing dataset from Kaggle

In [1]:
!pip install kaggle

In [4]:
!mkdir ~/.kaggle .kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
import json
token = {"username":"rikeadelia","key":"fea2c81cab3e56041c74635dd47645f9"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [6]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

- path is now set to: {/content}


In [7]:
!kaggle datasets download -d minhhuy2810/rice-diseases-image-dataset -p /content

100% 12.0G/12.0G [04:50<00:00, 57.9MB/s]
100% 12.0G/12.0G [04:50<00:00, 44.3MB/s]


In [0]:
!unzip rice-diseases-image-dataset.zip

In [0]:
!rm rice-diseases-image-dataset.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## The Model

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras

In [18]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/RiceDiseaseDataset/train/',  # This is the source directory for training images
        target_size=(128,128),
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/RiceDiseaseDataset/validation/',  # This is the source directory for training images
        target_size=(128,128),
        batch_size=8,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 1600 images belonging to 4 classes.
Found 492 images belonging to 4 classes.


In [21]:
model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 12, 12, 64)       

In [0]:
save_callback = keras.callbacks.ModelCheckpoint(
    "/content/drive/My Drive/bangkit/trial1.hdf5",
    monitor="val_accuracy",
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch"
)
logger = keras.callbacks.CSVLogger("/content/drive/My Drive/bangkit/log.csv")


In [0]:
history = model.fit(
      train_generator,
      epochs=30,
      verbose=1,
      validation_data = validation_generator,
    callbacks=[save_callback,logger]
)

Epoch 1/30
 2/25 [=>............................] - ETA: 1:00 - loss: 1.3829 - accuracy: 0.3125

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

# plt.show()
plt.savefig('/content/drive/My Drive/bangkit/plot.png')